In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import glob
import ntpath
from ipywidgets import Image, Layout, interact, interactive, fixed, interact_manual, widgets
from lasso.dyna import D3plot, ArrayType
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding, MDS

In [2]:
filepath_list = glob.glob("exercise-1/d3plots/*.d3plot")
print(f"Found {len(filepath_list)} files.")

Found 100 files.


In [3]:
# let's collect node displacement from all d3plots
fields = []

for i_filepath, filepath in enumerate(filepath_list):
    
    # print progress
    print(f"filepath {filepath} ({i_filepath + 1}/{len(filepath_list)})", end='\r')
    
    # load d3plot
    d3plot = D3plot(filepath)
    
    # extract node displacement at last timestep
    node_displacement = d3plot.arrays[ArrayType.node_displacement]
    fields.append(node_displacement)

# print end
print(f"Done ({i_filepath + 1}/{len(filepath_list)})")

Done (100/100)ise-1/d3plots\run99.d3plot (100/100)


In [4]:
# convert list into a numpy array
data = np.stack(fields)

n_simulations, n_timesteps, n_nodes, xyz = data.shape
print(f"(n_simulations {n_simulations}, n_timesteps {n_timesteps}, n_nodes {n_nodes}, xyz {xyz})")

(n_simulations 100, n_timesteps 5, n_nodes 2500, xyz 3)


In [5]:
# select some simulation
some_simulation = 0

# transform shape
data_sim = data[some_simulation].reshape((n_timesteps, n_nodes*xyz))

print(f"(n_timesteps {data_sim.shape[0]}, n_nodes*xyz flattened {data_sim.shape[1]})")

(n_timesteps 5, n_nodes*xyz flattened 7500)


In [6]:
u, s, v = np.linalg.svd(data_sim)

print(f"u {u.shape}")
print(f"s {s.shape}")
print(f"v {v.shape}")

u (5, 5)
s (5,)
v (7500, 7500)


In [9]:
# displace singular values/eigenvalues
fig = go.FigureWidget()
fig.add_trace(
    go.Scatter(x=np.arange(len(s)), 
               y=s, 
               mode='lines+markers',
               name="Singular Values"
              )
)

fig.update_layout(
    title='Singular Values',
    xaxis_title='Eigenvalue Index',
    yaxis_title='Eigenvalue',
)

fig.show()

In [15]:
i_timestep = 0
i_modes = 1

def reconstruct_simulation(i_modes, u, s, v):
    return np.einsum("tr,r,rn->tn", 
                     u[:,:i_modes], 
                     s[:i_modes], 
                     v[:i_modes, :]).reshape((n_timesteps, n_nodes, xyz))

data_reconstructed = reconstruct_simulation(i_modes, u, s, v)[i_timestep]

fig = go.FigureWidget(
    data=[
        go.Mesh3d(
            x=data_reconstructed[:, 0],
            y=data_reconstructed[:, 1],
            z=data_reconstructed[:, 2],
            #mode='markers',
            name="Embedding Simulation",
            #marker={
            #   "size": 5
            #},
            #text=filepath_list,
            #hoverinfo="text",
            #opacity=0.99
        )
    ]
)

# some styling 
fig.update_layout(title='3D Embedding of Simulations',
                  margin=go.layout.Margin(
                    l=0, #left margin
                    r=0, #right margin
                    b=0, #bottom margin
                    t=0  #top margin
                  ),
                  #hovermode="closest",
                  autosize=False,
                 )


def on_timestep_change(timestep_value):
    global i_timestep
    i_timestep = timestep_value - 1
    data_reconstructed[:] = reconstruct_simulation(i_modes, u, s, v)[i_timestep]
    fig.update_traces(overwrite=True, 
                      x=data_reconstructed[:, 0],
                      y=data_reconstructed[:, 1],
                      z=data_reconstructed[:, 2],)

def on_modes_change(mode_value):
    global i_modes
    i_modes = mode_value
    data_reconstructed[:] = reconstruct_simulation(i_modes, u, s, v)[i_timestep]
    fig.update_traces(overwrite=True, 
                      x=data_reconstructed[:, 0],
                      y=data_reconstructed[:, 1],
                      z=data_reconstructed[:, 2],)

interact(on_timestep_change, timestep_value=widgets.IntSlider(min=1, max=n_timesteps, step=1, value=1))
interact(on_modes_change, mode_value=widgets.IntSlider(min=1, max=len(s), step=1, value=1))

print(data_reconstructed.shape)


#from ipywidgets import HBox, VBox
#HBox([fig,
#      image_widget])
fig

interactive(children=(IntSlider(value=1, description='timestep_value', max=5, min=1), Output()), _dom_classes=…

interactive(children=(IntSlider(value=1, description='mode_value', max=5, min=1), Output()), _dom_classes=('wi…

(2500, 3)


FigureWidget({
    'data': [{'name': 'Embedding Simulation',
              'type': 'mesh3d',
              'ui…